In [2]:
# Python-Oracle 연동 - 입력, 수정, 삭제
## 01. 준비과정
### 1. 패키지 참조
import cx_Oracle as cx  # '0'에 대문자 주의

## 02.cx_Oracle을 사용한 연동
### 1. 데이터베이스 접속 및 cursor 객체 생성

In [4]:
dbcon = cx.connect("hr","hr","localhost:1521/xe")
cursor = dbcon.cursor()

### 2. 데이터 저장하기

In [6]:
dname = input("학과이름을 입력하세요")
loc = input("학과의 위치를 입력하세요")
dname, loc

("메가\\'IT\\'학과", '여기')

#### 2) 저장을 위한 새로운 시퀀스값 조회

In [7]:
sql = "select seq_department.nextval from dual"
cursor.execute(sql)
result = cursor.fetchall()
print(result)

seq = result[0][0]
print("사로운 시퀀스 번호: %d" % seq)

[(320,)]
사로운 시퀀스 번호: 320


#### 3) INSERT문 수행                                

In [8]:
# sql = "INSERT INTO department (deptno, dname, loc) values (seq_department.nextval from dual, "국문학과", ")"  TIP. SQL injection 공격?
sql = "INSERT INTO department (deptno, dname, loc) VALUES (:1,:2,:3)"

#                     :1    :2   :3
cursor.execute(sql, [seq, dname, loc])
print("%s개의 행이 저장됨" % cursor.rowcount)

# 처리 결과를 실제로 반영
dbcon.commit()
# 되돌리기 
 # 이미 commit()한 내용은 적용 안됨
 # rollback;

1개의 행이 저장됨


#### 4) UPDATE문 수행

In [9]:
sql = "Update department SET dname =:1, loc=:2 WHERE deptno=:3"

#                       :1          :2     :3
cursor.execute(sql, ['메가스터디', '신촌', seq])
print("%s개의 행이 갱신됨" % cursor.rowcount)
dbcon.commit()

1개의 행이 갱신됨


In [15]:
sql = "SELECT * FROM department"
cursor.execute(sql)
cursor.rowfactory = lambda * args: dict(zip([d[0]for d in cursor.description], args))

result = cursor.fetchall()
result

[{'DEPTNO': 320, 'DNAME': '메가스터디', 'LOC': '신촌'},
 {'DEPTNO': 101, 'DNAME': '컴퓨터공학과', 'LOC': '1호관'},
 {'DEPTNO': 102, 'DNAME': '멀티미디어학과', 'LOC': '2호관'},
 {'DEPTNO': 201, 'DNAME': '전자공학과', 'LOC': '3호관'},
 {'DEPTNO': 202, 'DNAME': '컴퓨터공학과', 'LOC': '공학관'},
 {'DEPTNO': 302, 'DNAME': '정보통신학과', 'LOC': '6호관'}]

#### 5) DELETE문 수행

In [16]:
sql = "DELETE from department where deptno = :1"
cursor.execute(sql, [seq])
print("%s개의 행이 삭제됨" % cursor.rowcount)
dbcon.commit()

1개의 행이 삭제됨


In [17]:
sql = "SELECT * FROM department"
cursor.execute(sql)
cursor.rowfactory = lambda * args: dict(zip([d[0]for d in cursor.description], args))

result = cursor.fetchall()
result

[{'DEPTNO': 101, 'DNAME': '컴퓨터공학과', 'LOC': '1호관'},
 {'DEPTNO': 102, 'DNAME': '멀티미디어학과', 'LOC': '2호관'},
 {'DEPTNO': 201, 'DNAME': '전자공학과', 'LOC': '3호관'},
 {'DEPTNO': 202, 'DNAME': '컴퓨터공학과', 'LOC': '공학관'},
 {'DEPTNO': 302, 'DNAME': '정보통신학과', 'LOC': '6호관'}]

### 3. 데이터베이스 접속 해제

In [18]:
try:
    cursor.close()
    dbcon.close()
except Exception as e:
    print("접속 상태가 아닙니다.", e)